In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

## Data Prep

In [2]:
df = pd.read_csv("../Phase1/main_data.csv", index_col=0)
df = df.dropna()
df

,# of bumps,# of inflections,curvature,min i,max i,x min,x max,y min,y max,S,criticality
0,3,0,3.255230,38,31,3.663562,3.465736,-11.639162,-10.540549,1.038058,1
1,0,0,1.404121,39,34,3.688879,3.555348,-11.066295,-10.373147,1.039465,1
2,0,0,-7.082191,6,0,1.945910,0.000000,-10.131818,-0.173747,1.063427,1
3,1,0,-8.379891,22,0,3.135494,0.000000,-10.842791,-0.665505,1.044607,1
4,1,0,-13.070105,36,36,3.610918,3.610918,-10.964433,-10.964433,1.041672,1
...,...,...,...,...,...,...,...,...,...,...,...
8793,0,0,-1.873302,59,63,5.823046,6.093570,-8.122181,-7.622121,1.077089,3
8794,0,0,-1.102228,53,65,5.472271,6.297109,-8.539737,-7.719609,1.077744,3
8795,1,0,-0.846527,67,71,7.021976,7.340836,-14.026634,-7.243576,1.106867,3
8796,0,0,-1.016451,66,67,6.200509,6.265301,-14.535209,-13.497222,1.073818,1


In [3]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

X = df.drop('criticality', axis=1).values
Y = df['criticality'].values
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, stratify=Y, test_size=0.2, random_state=42)
# shuffled_data = shuffle(df.values, random_state=0)
# shuffled_data.shape

# test_index = int(shuffled_data.shape[0] * 0.8)

# X_train, X_test = shuffled_data[:test_index, :-1], shuffled_data[test_index:, :-1]
# Y_train, Y_test = shuffled_data[:test_index, -1], shuffled_data[test_index:, -1]
# shuffled_data.shape
print(X_train.shape, X_test.shape)

(6568, 10) (1643, 10)


## Decision Tree

In [4]:
from sklearn import tree

In [9]:
%%timeit
dt_clf = tree.DecisionTreeClassifier(random_state=0, max_depth=10)
dt_clf.fit(X_train, Y_train)

np.sum(dt_clf.predict(X_test) == Y_test) / len(X_test)

133 ms ± 2.08 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## KNN (Neatest Neighbor Classification)
**This takes a little bit of time**

In [6]:
from sklearn import neighbors
from sklearn.pipeline import Pipeline

print(sorted(neighbors.KDTree.valid_metrics))

['chebyshev', 'cityblock', 'euclidean', 'infinity', 'l1', 'l2', 'manhattan', 'minkowski', 'p']


In [8]:
nca = neighbors.NeighborhoodComponentsAnalysis(random_state=42)
knn = neighbors.KNeighborsClassifier(n_neighbors=3)
nca_pipe = Pipeline([('nca', nca), ('knn', knn)])
nca_pipe.fit(X_train, Y_train)

Pipeline(steps=[('nca', NeighborhoodComponentsAnalysis(random_state=42)),
                ('knn', KNeighborsClassifier(n_neighbors=3))])

In [10]:
nca_pipe.score(X_test, Y_test)

0.8922702373706635

## Metrics
Couldn't think of any other mectrics

In [11]:
# confusion metrics
from sklearn.metrics import confusion_matrix

In [16]:
print("Decision Tree:\n", confusion_matrix(y_true=Y_test, y_pred=dt_clf.predict(X_test), normalize='true', labels=[1, 2, 3]))
print("\n\n")
print("NCA Pipeline:\n", confusion_matrix(y_true=Y_test, y_pred=nca_pipe.predict(X_test), normalize='true', labels=[1, 2, 3]))

Decision Tree:
 [[0.9132948  0.06936416 0.01734104]
 [0.31410256 0.53846154 0.1474359 ]
 [0.00929752 0.02892562 0.96177686]]



NCA Pipeline:
 [[0.9132948  0.05780347 0.02890173]
 [0.35897436 0.47435897 0.16666667]
 [0.02169421 0.02995868 0.94834711]]
